In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import datetime
from datetime import date
import random

/Users/polinab/.virtualenvs/base/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Part 1: Data Generation
- Since our DAPP is just a prototype, and the information we would track is not publicly available, we mocked up data
- this data does not take into account real-world distributions of immunization type or frequncies
    - however, we put half of the values as PCR tests since this is the most common form of immunity proof now and for the foreseeable future
- similarly, the travel destinations are dummy variables
- the data generation includes business logic for how to determine a travelers validity 
- this is also a good proxy for how the off-chain databases could be organized
- please refer to comments in the code for specific explanations of the logic if unclear from code itself

In [2]:
# this function generates n random dates within a range

def random_dates(start, end, l=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, l), unit='s')

In [3]:

#### IMMUNIZATION DATA ####

# lookup dict of immunization 3-character code and the full name
vax_desc = {'pcr':'PCR Test',
               'rec':'Proof Of Recovery',
               'jj':'Johnson & Johnson',
               'az1':'Astra-Zeneca Dose 1',
               'az2':'Astra-Zeneca Dose 2'}

# lookup dict of immunization 3-char code and the type of immunization
vax_id_type = {'pcr':'test',
               'rec':'rec',
               'jj':'vax',
               'az1':'vax1',
               'az2':'vax2'}

#  lookup dict of type of immunization and number of days of validity
vax_duration = {'test':3,'vax1':0,'vax2':365,'vax':365,'rec':180}

# unique list of all immunization 3-char codes
immunization_ids = [key for key,value in vax_id_type.items()]

In [4]:
### PATIENT DATA GENERATION ###

n_pa = 80 # number of total distinct patients in our sample
n = 100 # number of immunization records in our sample

patient_id_unique = random.sample(range(10000, 99999), n_pa)
patient_ids = random.choices(patient_id_unique, k=n) # patients can get vaxed multiple times, so we resample
patient_immunization_ids = ['pcr']*(n//2) +random.choices(immunization_ids, k=(n//2)) # pretending that at least 50% of immunity proofs are PCR tests


In [5]:
### MEDICAL CENTER DATA GENERATION ####

# dummy list of cities, can add to this as records come in. This way we can look up country easily
cities = {'San Francisco':'USA',
          'New York':'USA',
          'Barcelona':'Spain',
          'Paris':'France',
          'London':'UK',
          'Milan':'Italy',
          'Dublin':'Ireland'
         }

med_center_id_unique = random.sample(range(1000, 9999), 30) # 30 total medical centers, assigning unique codes
med_center_cities = random.choices([c for c in cities.keys()], k=30) # randomly assigning cities for each center
med_center_countries = [cities[c] for c in med_center_cities] # look up country by city name

med_cen_ids = random.sample(range(10000, 90000), n) # assigning medical centers from unique list to each vax record

In [6]:
### IMMUNIZATION RECORD DATA GENERATION ###

# setting start date for when immunizations began. This can be changed. End is current day
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime(date.today())

# randomly assigning a date within that range to every immunization
immunization_dates = random_dates(start, end,l=100)


In [11]:
# Immunization dimension table

vax_dims = {'id':immunization_ids,
            'name':[vax_desc[vid] for vid in immunization_ids],
            'type':[value for key,value in vax_id_type.items()],
            'immune_days':[vax_duration[key] for key in vax_id_type.values()]}

vax_dim_df = pd.DataFrame.from_dict(vax_dims)
vax_dim_df.head()

,id,name,type,immune_days
0,pcr,PCR Test,test,3
1,rec,Proof Of Recovery,rec,180
2,jj,Johnson & Johnson,vax,365
3,az1,Astra-Zeneca Dose 1,vax1,0
4,az2,Astra-Zeneca Dose 2,vax2,365


In [8]:
# immunization record table

vax_records = {'patient_id':patient_ids,
               'immunization_id': patient_immunization_ids,
               'immunization_date': immunization_dates,
               'medical_center_id': med_cen_ids}

vax_rec_df = pd.DataFrame.from_dict(vax_records)

# from the immunization length lookup, we set the data of validity of each immunization record 
vax_rec_df['immunization_validity_date']=vax_rec_df.apply(
    lambda x: x['immunization_date']+ datetime.timedelta(days=vax_duration[vax_id_type[x.immunization_id]]),axis=1)
vax_rec_df.head()

,patient_id,immunization_id,immunization_date,medical_center_id,immunization_validity_date
0,92856,pcr,2021-03-24 01:57:41,33774,2021-03-27 01:57:41
1,20942,pcr,2021-05-29 16:02:43,86737,2021-06-01 16:02:43
2,23772,pcr,2021-04-20 11:42:20,53828,2021-04-23 11:42:20
3,82603,pcr,2021-02-20 10:46:34,44391,2021-02-23 10:46:34
4,22318,pcr,2021-01-14 13:25:08,78613,2021-01-17 13:25:08


In [9]:

med_cen_records = {'med_center_id':med_center_id_unique,
                   'name':[f'Dummy Name {i}' for i in med_center_id_unique], # placeholder for actual names
                   'city': med_center_cities,
                   'country':med_center_countries}
med_cen_df = pd.DataFrame.from_dict(med_cen_records)
med_cen_df.head()


,med_center_id,name,city,country
0,8546,Dummy Name 8546,San Francisco,USA
1,2704,Dummy Name 2704,Paris,France
2,4909,Dummy Name 4909,San Francisco,USA
3,2322,Dummy Name 2322,London,UK
4,1177,Dummy Name 1177,Paris,France


In [13]:
### TRAVEL DATA GENERATION ###

kn = 50 # 50 trips, we can change this 

# create to:from list from our existing list of cities, omitting where to and from is the same
from_to_combos = [[i,j] for j in cities for i in cities if i!=j]

# take kn samples from this list of possible routes
from_to = random.choices(from_to_combos, k=kn)


travel_records = {'customer_id':random.choices(patient_ids, k=kn), # choose from existing list of patients
                  'travel_date':random_dates(start, end,l=kn),
                  'from_city':[e[0] for e in from_to],
                  'to_city':[e[1] for e in from_to]
                 }

travel_df = pd.DataFrame.from_dict(travel_records)

# business logic for checking immunity status: find latest valid immunity date for each traveler
# check if that immunity date is before or after the travel date
travel_df['immunity_end_date'] = travel_df.apply(
    lambda x: vax_rec_df[vax_rec_df.patient_id == x.customer_id].immunization_validity_date.max(),axis=1)
travel_df['immunity_status'] = travel_df.apply(lambda x: True if x.immunity_end_date > x.travel_date else False, axis=1)
travel_df.head()


,customer_id,travel_date,from_city,to_city,immunity_end_date,immunity_status
0,16052,2021-02-16 03:36:22,Barcelona,Dublin,2021-09-24 15:37:57,True
1,61641,2021-05-22 20:00:38,Milan,Paris,2021-03-29 13:57:57,False
2,10443,2021-05-31 21:48:52,Paris,New York,2021-08-13 07:51:08,True
3,43056,2021-02-12 19:16:01,San Francisco,New York,2021-09-08 12:57:01,True
4,85995,2021-03-16 18:41:39,Dublin,Milan,2021-05-01 07:52:02,True
